# LAB 12 - Random Forest for Regression

In this lab we will be extending the previous lab about Decision trees and build a Regression model using Random Forest.

For simplicity, we will be using the same dataset as the previous lab (you can find it in ECLASS).

**IMPORTANT:** For this lab, if you haven't finished your code from last week's lab on Decision trees, you will have the option to use the sklearn implementation for a regression tree. However, this doesn't mean that you should skip the previous lab. This is just so that you don't get behind with the content and you don't spend all your time today working on the previous lab. 

In [21]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

As mentioned before, use the Boston Housing data and prepare your train/val/test split as usual.

## Exercise 1 -- Bootstrap

Also known as [bagging](https://en.wikipedia.org/wiki/Bootstrap_aggregating), this technique consists of making several samples with replacement of the original data, using each of the samples to train an estimator, and then aggregating the predictions using the average (this is also a type of model ensemble).

In [22]:
def bootstrap(X, num_bags=10):
    """
    Given a dataset and a number of bags,
    sample the dataset with replacement.
    
    This function does not return a copy
    of the datapoints, but a list of indices
    with compatible dimensionality
    
    Parameters
    ----------
    X : ndarray
        A dataset
    num_bags : int, default 10
        The number of bags to create
    
    Returns
    -------
    list of ndarray
        The list contains `num_bags` integer one-dimensional ndarrays.
        Each of these contains the indices corresponding to the 
        sampled datapoints in `X`
    
    Notes
    -----
    * The number of datapoints in each bach will
      match the number of datapoints in the given
      dataset.
    * The
    """
    rng = np.random.default_rng(0) # you can change the seed, or use 0 to replicate my results
    
    n = X.shape[0]

    bags = []
    
    for _ in range(num_bags):
        bags.append(rng.integers(0, n, n))
    
    return np.array(bags)

In [23]:
rng = np.random.default_rng(0)
X_small = rng.random(size=(100,2))
bags = bootstrap(X_small)
bags[0]

array([85, 63, 51, 26, 30,  4,  7,  1, 17, 81, 64, 91, 50, 60, 97, 72, 63,
       54, 55, 93, 27, 81, 67,  0, 39, 85, 55,  3, 76, 72, 84, 17,  8, 86,
        2, 54,  8, 29, 48, 42, 40,  2,  0, 12,  0, 67, 52, 64, 25, 61, 76,
       38, 46, 99, 80, 98, 37, 68, 95, 65, 84, 68, 70, 38, 87, 13, 57, 72,
       84, 52, 37, 31, 42, 48, 71, 88,  7, 93, 53, 35, 67, 57, 25, 32, 71,
       59, 50, 33, 76, 39, 32, 89, 26, 22, 71, 62,  4,  8, 37, 83])

## Exercise 2 -- Aggregation

The second part of bagging.

In [24]:
def aggregate_regression(preds):
    """
    Aggregate predictions by several estimators
    
    Parameters
    ----------
    preds : list of ndarray
        Predictions from multiple estimators.
        All ndarrays in this list should have the same
        dimensionality.
        
    Return
    ------
    ndarray
        The mean of the predictions
    """
    estimators = np.array(preds)

    return estimators.mean(axis=0)

## Exercise 3 -- Random Forest for regression

Using the functions you implemented above, it is now time to put all of them together to train several decision trees and then ensemble them to output a single prediction. For the random forest, however, we need to select a subset of features at each split on the decision tree. 

For this part, you can use the sklearn implementation of Random forest for regression as your estimator for each set of features and bags. See below an example of how to do this, and always remember to check the necessary documentation when using an external function.

Some parameters you will have to set are: 
* num_features: number of features per estimator
* min_samples: min number of samples per leaf node
* max_depth: maximum depth of the decision tree (each estimator)
* num_estimators: number of decision trees you will create using each bag and random set of features

In [ ]:
columns = ["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT","MEDV"]
boston = pd.read_csv("housing.txt", sep=r'\s+', header=None)
boston.columns = columns

boston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [26]:
X = boston.drop("MEDV", axis=1)
y = boston["MEDV"]

In [27]:
X

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48


In [28]:
y

0      24.0
1      21.6
2      34.7
3      33.4
4      36.2
       ... 
501    22.4
502    20.6
503    23.9
504    22.0
505    11.9
Name: MEDV, Length: 506, dtype: float64

In [29]:
max_depth = 3
num_features = 8
min_samples = 10
num_estimators = X.shape[0]

In [71]:
# List containing the OOB samples of each bootstrap
OOB_X_samples = []
OOB_y_samples = []

estimators = []  # List of all trees that I create
samples_idx = bootstrap(X, num_estimators)  # The matrix of indexes
                                            # Each line represents a bootstrap

for i in range(num_estimators):
    X_bootstrap = X.iloc[samples_idx[i]]  # Get the bootstrap sample
    y_bootstrap = y[samples_idx[i]]  # Get the bootstrap sample responses

    OOB_X_samples.append(X.drop(samples_idx[i]))  # Getting the OOB of each bootstrap
    OOB_y_samples.append(y.drop(samples_idx[i]))  # Getting the corresponding targets for each OOB

    # Creating and fitting the estimator
    estimator = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples, max_features=num_features)
    estimator.fit(X_bootstrap, y_bootstrap)

    estimators.append(estimator)

In [ ]:
errors = []

# Calculating how good the random forest is (OOB Error)
for j, sample in enumerate(OOB_X_samples):
    predictions = []  # Creating the list of predictions
    for estimator in estimators:  # For each tree in the forest
        y_pred = estimator.predict(sample)  # I calculate the prediction for that sample
        predictions.append(y_pred)  # And add it on the predictions list
    
    final_y_pred = aggregate_regression(predictions)  # Aggregating all the results

    errors.append((final_y_pred - OOB_y_samples[j])**2)

In [77]:
oob_err_for_each_datapoint = np.zeros(shape=(X.shape[0]))

for n in range(X.shape[0]):
    counter = 0
    for j, sample in enumerate(OOB_X_samples):
        if n not in sample.index:  # Checks if the datapoint is a OOB datapoint for the bootstrap being analysed
            continue
        
        counter += 1
        oob_err_for_each_datapoint[n] += errors[j][n]
    oob_err_for_each_datapoint[n] /= counter

In [79]:
oob_err_for_each_datapoint.mean()

np.float64(13.104716165341102)